In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils, to_categorical

2022-11-23 09:34:20.034242: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-23 09:34:20.587312: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-23 09:34:20.587396: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-23 09:34:22.914791: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-

In [3]:
df = pd.read_csv("database_mscc.csv")

In [4]:
df.drop(columns = "Unnamed: 0", inplace=True)

In [5]:
for row in np.arange(0,df.shape[0]):
    df["y"][row] = df["y"][row][7:]

/tmp/ipykernel_4583/4108488883.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["y"][row] = df["y"][row][7:]


In [6]:
df.isna().sum()

y         0
0         0
1         0
2         0
3         0
       ... 
224    1439
225    1439
226    1439
227    1439
228    1440
Length: 230, dtype: int64

In [7]:
df = df.fillna(0)

In [8]:
df["y"].drop_duplicates()

0     surprised
1       fearful
2       disgust
14        happy
15         calm
18      neutral
34          sad
35        angry
Name: y, dtype: object

In [9]:
y = df["y"]
X = df.drop(columns="y")

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=1405, stratify=y)

In [11]:
lb = LabelEncoder()
y_train_cat = np_utils.to_categorical(lb.fit_transform(y_train))
y_test_cat = np_utils.to_categorical(lb.fit_transform(y_test))

In [12]:
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [13]:
X_train_scaled_array = np.array(X_train_scaled)
X_test_scaled_array = np.array(X_test_scaled)
y_train_array = np.array(y_train_cat)
y_test_array = np.array(y_test_cat)

In [14]:
X_train_expanded_dims = np.expand_dims(X_train_scaled_array, axis=2)
X_test_expanded_dims = np.expand_dims(X_test_scaled_array, axis=2)

In [15]:
model = models.Sequential()
model.add(layers.Conv1D(256, 8, padding="same", input_shape=(X_train_expanded_dims.shape[1],1), activation = "tanh"))
model.add(layers.Conv1D(256, 8, padding='same', activation = "relu"))
# model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.25))
model.add(layers.MaxPooling1D(pool_size=(8)))
model.add(layers.Conv1D(128, 8, padding='same', activation = "relu"))
model.add(layers.Conv1D(128, 8, padding='same', activation = "relu"))
model.add(layers.Conv1D(128, 8, padding='same', activation = "relu"))
model.add(layers.Conv1D(128, 8, padding='same', activation = "relu"))
# model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.25))
model.add(layers.MaxPooling1D(pool_size=(8)))
model.add(layers.Conv1D(64, 8, padding='same', activation = "relu"))
model.add(layers.Conv1D(64, 8, padding='same', activation = "relu"))
model.add(layers.Flatten())
model.add(layers.Dense(8, activation = "softmax"))
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

2022-11-23 09:36:04.316327: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-23 09:36:04.316445: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-23 09:36:04.316491: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-23E8V41): /proc/driver/nvidia/version does not exist
2022-11-23 09:36:04.322241: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 229, 256)          2304      
                                                                 
 conv1d_1 (Conv1D)           (None, 229, 256)          524544    
                                                                 
 dropout (Dropout)           (None, 229, 256)          0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 28, 256)          0         
 )                                                               
                                                                 
 conv1d_2 (Conv1D)           (None, 28, 128)           262272    
                                                                 
 conv1d_3 (Conv1D)           (None, 28, 128)           131200    
                                                        

In [30]:
es = EarlyStopping(patience=10, restore_best_weights=True)
model.fit(X_train_expanded_dims, y_train_array, validation_split=0.3, epochs=500, batch_size=32, callbacks=[es])

Epoch 1/500
23/23 [==============================] - 12s 508ms/step - loss: 0.8195 - accuracy: 0.6738 - val_loss: 1.9986 - val_accuracy: 0.3267
Epoch 2/500
23/23 [==============================] - 11s 470ms/step - loss: 0.8041 - accuracy: 0.6879 - val_loss: 2.0805 - val_accuracy: 0.3333
Epoch 3/500
23/23 [==============================] - 10s 451ms/step - loss: 1.1570 - accuracy: 0.5702 - val_loss: 1.9721 - val_accuracy: 0.3168
Epoch 4/500
23/23 [==============================] - 10s 439ms/step - loss: 0.9046 - accuracy: 0.6482 - val_loss: 2.0019 - val_accuracy: 0.3498
Epoch 5/500
23/23 [==============================] - 11s 460ms/step - loss: 0.7372 - accuracy: 0.7135 - val_loss: 2.2820 - val_accuracy: 0.3168
Epoch 6/500
23/23 [==============================] - 13s 552ms/step - loss: 0.6203 - accuracy: 0.7574 - val_loss: 2.3560 - val_accuracy: 0.3597
Epoch 7/500
23/23 [==============================] - 12s 515ms/step - loss: 1.6195 - accuracy: 0.4057 - val_loss: 1.9208 - val_accuracy:

In [31]:
model.evaluate(X_test_expanded_dims, y_test_array)

14/14 [==============================] - 2s 112ms/step - loss: 2.0218 - accuracy: 0.2917


[2.021777629852295, 0.2916666567325592]